#### 集成Ensumle分类模型是综合考量多个分类器的预测结果，从而做出决策。“综合考量”的方式大体有两种。
##### a.使用相同的训练数据，同时搭建多个独立的分类模型，然后以投票的方式，以少数服从多数的原则做出最终的分类决策。
##### 比较有代表性的是随机森林算法。但是一株标准的决策树会根据每维特征对预测结果的影响程度进行排序，
##### 进而决定不同特征从上至下构建分裂节点的顺序。
#### 这样的话，每次构建的都是同一棵树，为了避免这一点，随机深林算法放弃固有的排序算法，转而随机选取特征，这也是随机森林名字的由来

##### b.另一种是按照一定的次序搭建多个分类模型，这些模型彼此存在相互依赖关系，每一个后续模型的加入都需要对当前的模型综合性能有所贡献。
#####     进而不断提升更新过后的模型的性能。这一类的代表是梯度提升树Gradient Tree Boosting。与随机森林不同的是，这里面每一棵树在生成的过程中
#####     都会尽可能降低整体模型在训练集上的拟合误差

## 使用随机深林和梯度提升树继续对泰坦尼克号沉船事件数据进行分类预测

### 1.导入数据

In [1]:
import pandas as pd
df = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')

In [2]:
#查看数据基本信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


### 2.使用pclass,age,sex来预测船员的生存状况

In [3]:
X = df[['pclass','sex','age']]
Y = df['survived']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
sex       1313 non-null object
age       633 non-null float64
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


### 3.数据预处理

In [4]:
#查看信息发现，age只有633个是不为nan的，需要补充。使用均值或中位数补充。对于sex和pclass是类别类型，需要转换成数值特诊
X['age'].fillna(X['age'].mean(),inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
sex       1313 non-null object
age       1313 non-null float64
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


D:\softwares\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### 4.数据分割

In [5]:
#现在age已经补充完成，接下来要把sex、pclass壮观成数值
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=124)

D:\softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 5.类别数据转换

In [6]:
# 使用sklean的feature_extraction对特征进行转换
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(x_train.to_dict(orient='record'))
X_test = vec.fit_transform(x_test.to_dict(orient='record'))

### 6.使用普通决策树

In [18]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)
dtc_result = dtc.predict(X_test)

### 7.使用随机深林算法

In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

### 8.使用梯度提升树

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
gbc_predict = gbc.predict(X_test)

### 9.性能检测

In [21]:
from sklearn.metrics import classification_report
print('DecisionTreeClassifier预测准确度：\n',dtc.score(X_test,y_test))
print('DecisionTreeClassifier性能指标：\n',classification_report(dtc_result,y_test,target_names=['died','survived']))
print('RandomForestClassifier预测准确度：\n',rfc.score(X_test,y_test))
print('RandomForestClassifier性能指标：\n',classification_report(rfc_predict,y_test,target_names=['died','survived']))
print('GradientBoostingClassifier预测准确度：\n',gbc.score(X_test,y_test))
print('GradientBoostingClassifier性能指标：\n',classification_report(gbc_predict,y_test,target_names=['died','survived']))

DecisionTreeClassifier预测准确度：
 0.80547112462
DecisionTreeClassifier性能指标：
              precision    recall  f1-score   support

       died       0.90      0.83      0.86       242
   survived       0.61      0.74      0.67        87

avg / total       0.82      0.81      0.81       329

RandomForestClassifier预测准确度：
 0.799392097264
RandomForestClassifier性能指标：
              precision    recall  f1-score   support

       died       0.88      0.83      0.86       236
   survived       0.63      0.71      0.67        93

avg / total       0.81      0.80      0.80       329

GradientBoostingClassifier预测准确度：
 0.829787234043
GradientBoostingClassifier性能指标：
              precision    recall  f1-score   support

       died       0.93      0.84      0.88       248
   survived       0.62      0.80      0.70        81

avg / total       0.85      0.83      0.84       329



### 10.特点分析

##### 集成模型在实战中较为常见，它在训练的过程中需要耗费更多的时间，但是得到的模型往往有更高的表现性能和更好的稳定性